## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.


- **Model**: `zero_shot_ner`
- **Model Description**: This is a Named Entity Recognition (NER) model capable of identifying any entity type using a bidirectional transformer encoder (BERT-like). It utilizes input text and labels to predict named entities within the text.

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "zero-shot-ner"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"


### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

--------!

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/out{i}.{input_format}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/out{i}.{input_format}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

## Inputs

- **text**: Input text or list of texts to predict named entities.
- **labels**: 
  - Domain-specific labels: Provide a string specifying the domain from the table below.
  - Custom labels: Provide a list of labels or a single label as a string.
  - If no value is provided, the default domain will be set to clinical.
- **threshold**: Threshold for predictions (default is 0.5).


Domain-specific labels

| Domain         | Description                              | Entities                                                                                                                                                     |
|----------------|------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------|
| oncology       | Oncology-Specific Entities                   | Adenopathy, Age, Biomarker, Biomarker_Result, Cancer_Dx, Cancer_Score, Cancer_Surgery, Chemotherapy, Cycle_Count, Cycle_Day, Cycle_Number, Date, Death_Entity, Direction, Dosage, Duration, Frequency, Gender, Grade, Histological_Type, Hormonal_Therapy, Imaging_Test, Immunotherapy, Invasion, Line_Of_Therapy, Metastasis, Oncogene, Pathology_Result, Pathology_Test, Performance_Status, Race_Ethnicity, Radiation_Dose, Radiotherapy, Relative_Date, Response_To_Treatment, Route, Site_Bone, Site_Brain, Site_Breast, Site_Liver, Site_Lung, Site_Lymph_Node, Site_Other_Body_Part, Smoking_Status, Staging, Targeted_Therapy, Tumor_Finding, Tumor_Size, Unspecific_Therapy |
| radiology      | Radiology-related terms                  | BodyPart, Direction, Disease_Syndrome_Disorder, ImagingFindings, ImagingTest, Imaging_Technique, ManualFix, Measurements, Medical_Device, OtherFindings, Procedure, Symptom, Test, Test_Result, Units                                                                                       |
| sdoh           | Social determinants of health-related terms | Access_To_Care, Age, Alcohol, Childhood_Event, Communicable_Disease, Community_Safety, Diet, Disability, Eating_Disorder, Education, Employment, Environmental_Condition, Exercise, Family_Member, Financial_Status, Food_Insecurity, Gender, Geographic_Entity, Healthcare_Institution, Housing, Hyperlipidemia, Hypertension, Income, Insurance_Status, Language, Legal_Issues, Marital_Status, Mental_Health, Obesity, Other_Disease, Other_SDoH_Keywords, Population_Group, Quality_Of_Life, Race_Ethnicity, Sexual_Activity, Sexual_Orientation, Smoking, Social_Exclusion, Social_Support, Spiritual_Beliefs, Substance_Duration, Substance_Frequency, Substance_Quantity, Substance_Use, Transportation, Violence_Or_Abuse |
| nihss          | National Institutes of Health Stroke Scale (NIHSS) | 10_Dysarthria, 11_ExtinctionInattention, 1a_LOC, 1b_LOCQuestions, 1c_LOCCommands, 2_BestGaze, 3_Visual, 4_FacialPalsy, 5_Motor, 5a_LeftArm, 5b_RightArm, 6_Motor, 6a_LeftLeg, 6b_RightLeg, 7_LimbAtaxia, 8_Sensory, 9_BestLanguage, Measurement, NIHSS                                                                                                  |
| ade            | Adverse Drug Events (ADE)                | ADE, DRUG                                                                                                                                                   |
| clinical            | Clinical Entities                 | Admission_Discharge, Age, Alcohol, Allergen, BMI, Birth_Entity, Blood_Pressure, Cerebrovascular_Disease, Clinical_Dept, Communicable_Disease, Date, Death_Entity, Diabetes, Diet, Direction, Disease_Syndrome_Disorder, Dosage, Drug_BrandName, Drug_Ingredient, Duration, EKG_Findings, Employment, External_body_part_or_region, Family_History_Header, Fetus_NewBorn, Form, Frequency, Gender, HDL, Heart_Disease, Height, Hyperlipidemia, Hypertension, ImagingFindings, Imaging_Technique, Injury_or_Poisoning, Internal_organ_or_component, Kidney_Disease, LDL, Labour_Delivery, Medical_Device, Medical_History_Header, Modifier, O2_Saturation, Obesity, Oncological, Overweight, Oxygen_Therapy, Pregnancy, Procedure, Psychological_Condition, Pulse, Race_Ethnicity, Relationship_Status, RelativeDate, RelativeTime, Respiration, Route, Section_Header, Sexually_Active_or_Sexual_Orientation, Smoking, Social_History_Header, Strength, Substance, Substance_Quantity, Symptom, Temperature, Test, Test_Result, Time, Total_Cholesterol, Treatment, Triglycerides, VS_Finding, Vaccine, Vaccine_Name, Vital_Signs_Header, Weight |
| posology       | Drug Information related terms                   | DOSAGE, DRUG, DURATION, FORM, FREQUENCY, ROUTE, STRENGTH                                                                                                    |
| clinical_events       | Clinical events                           | PROBLEM, TEST, TREATMENT                                                                                                                                    |
| diseases       | Disease-related terms                    | Disease                                                                                                                                                     |
| chexpert       | Anatomical and Observation Entities in Chest Radiology Reports (CheXpert)                   | ANAT, OBS                                                                                                                                                   |
| risk_factors   | Risk factors-related terms               | CAD, DIABETES, FAMILY_HIST, HYPERLIPIDEMIA, HYPERTENSION, MEDICATION, OBESE, PHI, SMOKER                                                                   |
| bionlp         | Biology and genetics terms                     | Amino_acid, Anatomical_system, Cancer, Cell, Cellular_component, Developing_anatomical_structure, Gene_or_gene_product, Immaterial_anatomical_entity, Multi-tissue_structure, Organ, Organism, Organism_subdivision, Organism_substance, Pathological_formation, Simple_chemical, Tissue |
| anatomy        | Anatomy-related terms                    | Anatomical_system, Cell, Cellular_component, Developing_anatomical_structure, Immaterial_anatomical_entity, Multi-tissue_structure, Organ, Organism_subdivision, Organism_substance, Pathological_formation, Tissue                    |
| living_species | Living species-related terms             | HUMAN, SPECIES                                                                                                                                              |
| pathogen       | Pathogen, Medical Condition and Medicine related terms                   | MedicalCondition, Medicine, Pathogen                                                                                                                       |
> If you want to use domain-specific labels, provide the corresponding domain name from the table above in the `labels` field.


### Initial setup

In [18]:
ade_docs = [
    '''Always tired, and possible blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldn't find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the pain, I will pop a tylonol instead.''',
    '''I have an allergic reaction to vancomycin so I have itchy skin, sore throat/burning/itching, numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.'''
]

radiology_docs = [
        """The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, Xenaderm to wounds topically b.i.d., Lantus 40 units subcutaneously at bedtime, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. daily, Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., Percocet 5/325 mg 2 tablets q.4 h. p.r.n., magnesium citrate 1 bottle p.o. p.r.n., sliding scale coverage insulin, Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d.""",

        """Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma."""
]

sample_text = "I have an allergic reaction  to vancomycin  so I have itchy skin , sore throat/burning/itching , numbness of tongue  and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication."

### JSON

#### Example 1

In [19]:
ade_json = {"text": ade_docs,
            "labels": "ade"}

data =  process_data_and_invoke_realtime_endpoint(ade_json, content_type="application/json" , accept="application/json" )
pd.DataFrame(data["predictions"])

,document,ner_chunk,begin,end,ner_label,confidence
0,"Always tired, and possible blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldn't find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the pain, I will pop a tylonol instead.",blood clots,27,38,ADE,0.622143
1,"Always tired, and possible blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldn't find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the pain, I will pop a tylonol instead.",Voltaren,49,57,DRUG,0.990707
2,"Always tired, and possible blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldn't find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the pain, I will pop a tylonol instead.",blood clots,125,136,ADE,0.748520
3,"Always tired, and possible blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldn't find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the pain, I will pop a tylonol instead.",voltaren,302,310,DRUG,0.958548
4,"Always tired, and possible blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldn't find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the pain, I will pop a tylonol instead.",tylonol,544,551,DRUG,0.855309
5,"I have an allergic reaction to vancomycin so I have itchy skin, sore throat/burning/itching, numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",allergic reaction,10,27,ADE,0.930753
6,"I have an allergic reaction to vancomycin so I have itchy skin, sore throat/burning/itching, numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",vancomycin,31,41,DRUG,0.998339
7,"I have an allergic reaction to vancomycin so I have itchy skin, sore throat/burning/itching, numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",itchy skin,52,62,ADE,0.924099
8,"I have an allergic reaction to vancomycin so I have itchy skin, sore throat/burning/itching, numbness of tongue a

#### Example 2

In [20]:
radiology_json = {"text": radiology_docs,
                    "labels": "radiology"}

data = process_data_and_invoke_realtime_endpoint(radiology_json, content_type="application/json" , accept="application/json")
pd.DataFrame(data["predictions"])

,document,ner_chunk,begin,end,ner_label,confidence
0,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, Xenaderm to wounds topically b.i.d., Lantus 40 units subcutaneously at bedtime, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. daily, Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., Percocet 5/325 mg 2 tablets q.4 h. p.r.n., magnesium citrate 1 bottle p.o. p.r.n., sliding scale coverage insulin, Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d.",diabetes,77,85,Disease_Syndrome_Disorder,0.604111
1,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, Xenaderm to wounds topically b.i.d., Lantus 40 units subcutaneously at bedtime, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. daily, Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., Percocet 5/325 mg 2 tablets q.4 h. p.r.n., magnesium citrate 1 bottle p.o. p.r.n., sliding scale coverage insulin, Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d.",type 2,87,93,Disease_Syndrome_Disorder,0.698794
2,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. Sh

#### Example 3

In [21]:
sample_text_json = {"text": sample_text,
                    "labels": ['age', 'symptom', 'diagnosis', 'problem', 'gender', 'procedure', 'test', 'body part', 'section header', 'biomarker', 'measurements', 'unit']}

data = process_data_and_invoke_realtime_endpoint(sample_text_json, content_type="application/json" , accept="application/json")
pd.DataFrame(data["predictions"])

,document,ner_chunk,begin,end,ner_label,confidence
0,"I have an allergic reaction to vancomycin so I have itchy skin , sore throat/burning/itching , numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",allergic reaction,10,27,diagnosis,0.687882
1,"I have an allergic reaction to vancomycin so I have itchy skin , sore throat/burning/itching , numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",itchy,54,59,symptom,0.994860
2,"I have an allergic reaction to vancomycin so I have itchy skin , sore throat/burning/itching , numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",skin,60,64,body part,0.995819
3,"I have an allergic reaction to vancomycin so I have itchy skin , sore throat/burning/itching , numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",sore,67,71,symptom,0.984152
4,"I have an allergic reaction to vancomycin so I have itchy skin , sore throat/burning/itching , numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",throat,72,78,body part,0.985427
5,"I have an allergic reaction to vancomycin so I have itchy skin , sore throat/burning/itching , numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",burning,79,86,symptom,0.989564
6,"I have an allergic reaction to vancomycin so I have itchy skin , sore throat/burning/itching , numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",itching,87,94,symptom,0.994643
7,"I have an allergic reaction to vancomycin so I have itchy skin , sore throat/burning/itching , numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",numbness,97,105,symptom,0.957343
8,"I have an allergic reaction to vancomycin so I have itchy skin , sore throat/burning/itching , numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",tongue,109,115,body part,0.944688
9,"I have an allergic reaction to vancomycin so I have itchy skin , sore throat/burning/itching , numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",gums,121,125,body part,0.988204


### JSON Lines

In [22]:
import json

def create_jsonl(records, label):
    json_records = []

    for text in records:
        record = {
            "text": text,
            "labels": label
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

ade_jsonl = create_jsonl(ade_docs, "ade")
radiology_jsonl = create_jsonl(radiology_docs, "radiology")
sample_text_jsonl = create_jsonl([sample_text], ['age', 'symptom', 'diagnosis', 'problem', 'gender', 'procedure', 'test', 'body part', 'section header', 'biomarker', 'measurements', 'unit'])

#### Example 1

In [23]:
data = process_data_and_invoke_realtime_endpoint(ade_jsonl, content_type='application/jsonlines' , accept='application/jsonlines')
print(data)

{"predictions": [{"ner_chunk": "blood clots", "begin": 27, "end": 38, "ner_label": "ADE", "confidence": 0.6221432089805603}, {"ner_chunk": "Voltaren", "begin": 49, "end": 57, "ner_label": "DRUG", "confidence": 0.9907069206237793}, {"ner_chunk": "blood clots", "begin": 125, "end": 136, "ner_label": "ADE", "confidence": 0.7485202550888062}, {"ner_chunk": "voltaren", "begin": 302, "end": 310, "ner_label": "DRUG", "confidence": 0.9585478901863098}, {"ner_chunk": "tylonol", "begin": 544, "end": 551, "ner_label": "DRUG", "confidence": 0.8553090691566467}]}
{"predictions": [{"ner_chunk": "allergic reaction", "begin": 10, "end": 27, "ner_label": "ADE", "confidence": 0.9307523965835571}, {"ner_chunk": "vancomycin", "begin": 31, "end": 41, "ner_label": "DRUG", "confidence": 0.998339056968689}, {"ner_chunk": "itchy skin", "begin": 52, "end": 62, "ner_label": "ADE", "confidence": 0.9240986704826355}, {"ner_chunk": "sore throat", "begin": 64, "end": 75, "ner_label": "ADE", "confidence": 0.907771229

#### Example 2

In [24]:
data = process_data_and_invoke_realtime_endpoint(radiology_jsonl, content_type='application/jsonlines' , accept='application/jsonlines')
print(data)

{"predictions": [{"ner_chunk": "diabetes", "begin": 77, "end": 85, "ner_label": "Disease_Syndrome_Disorder", "confidence": 0.6041108965873718}, {"ner_chunk": "type 2", "begin": 87, "end": 93, "ner_label": "Disease_Syndrome_Disorder", "confidence": 0.6987941265106201}, {"ner_chunk": "coronary artery disease", "begin": 95, "end": 118, "ner_label": "Disease_Syndrome_Disorder", "confidence": 0.8235759735107422}, {"ner_chunk": "chronic renal insufficiency", "begin": 120, "end": 147, "ner_label": "Disease_Syndrome_Disorder", "confidence": 0.8366949558258057}, {"ner_chunk": "peripheral vascular disease", "begin": 149, "end": 176, "ner_label": "Disease_Syndrome_Disorder", "confidence": 0.8868952393531799}, {"ner_chunk": "diabetes", "begin": 196, "end": 204, "ner_label": "Disease_Syndrome_Disorder", "confidence": 0.9246309399604797}, {"ner_chunk": "paraplegia", "begin": 287, "end": 297, "ner_label": "Disease_Syndrome_Disorder", "confidence": 0.5937107801437378}, {"ner_chunk": "lower extremities

#### Example 3

In [25]:
data = process_data_and_invoke_realtime_endpoint(sample_text_jsonl, content_type='application/jsonlines' , accept='application/jsonlines')
print(data)

{"predictions": [{"ner_chunk": "allergic reaction", "begin": 10, "end": 27, "ner_label": "diagnosis", "confidence": 0.6878823637962341}, {"ner_chunk": "itchy", "begin": 54, "end": 59, "ner_label": "symptom", "confidence": 0.9948595762252808}, {"ner_chunk": "skin", "begin": 60, "end": 64, "ner_label": "body part", "confidence": 0.995819091796875}, {"ner_chunk": "sore", "begin": 67, "end": 71, "ner_label": "symptom", "confidence": 0.98415207862854}, {"ner_chunk": "throat", "begin": 72, "end": 78, "ner_label": "body part", "confidence": 0.9854265451431274}, {"ner_chunk": "burning", "begin": 79, "end": 86, "ner_label": "symptom", "confidence": 0.9895642995834351}, {"ner_chunk": "itching", "begin": 87, "end": 94, "ner_label": "symptom", "confidence": 0.9946433305740356}, {"ner_chunk": "numbness", "begin": 97, "end": 105, "ner_label": "symptom", "confidence": 0.9573434591293335}, {"ner_chunk": "tongue", "begin": 109, "end": 115, "ner_label": "body part", "confidence": 0.9446883797645569}, {"

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [26]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [27]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name1 = "input1.json"
validation_json_file_name2 = "input2.json"
validation_json_file_name3 = "input3.json"

validation_jsonl_file_name1 = "input1.jsonl"
validation_jsonl_file_name2 = "input2.jsonl"
validation_jsonl_file_name3 = "input3.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

# Write and upload JSON files
write_and_upload_to_s3(ade_json, f"{json_input_dir}/{validation_json_file_name1}")
write_and_upload_to_s3(radiology_json, f"{json_input_dir}/{validation_json_file_name2}")
write_and_upload_to_s3(sample_text_json, f"{json_input_dir}/{validation_json_file_name3}")


write_and_upload_to_s3(ade_jsonl, f"{jsonl_input_dir}/{validation_jsonl_file_name1}")
write_and_upload_to_s3(radiology_jsonl, f"{jsonl_input_dir}/{validation_jsonl_file_name2}")
write_and_upload_to_s3(sample_text_jsonl, f"{jsonl_input_dir}/{validation_jsonl_file_name3}")


### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [29]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data["predictions"])
    display(df)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

#### Example 1

In [30]:
process_s3_json_output_and_save(validation_json_file_name1)

,document,ner_chunk,begin,end,ner_label,confidence
0,"Always tired, and possible blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldn't find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the pain, I will pop a tylonol instead.",blood clots,27,38,ADE,0.622143
1,"Always tired, and possible blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldn't find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the pain, I will pop a tylonol instead.",Voltaren,49,57,DRUG,0.990707
2,"Always tired, and possible blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldn't find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the pain, I will pop a tylonol instead.",blood clots,125,136,ADE,0.748520
3,"Always tired, and possible blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldn't find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the pain, I will pop a tylonol instead.",voltaren,302,310,DRUG,0.958548
4,"Always tired, and possible blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldn't find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the pain, I will pop a tylonol instead.",tylonol,544,551,DRUG,0.855309
5,"I have an allergic reaction to vancomycin so I have itchy skin, sore throat/burning/itching, numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",allergic reaction,10,27,ADE,0.930753
6,"I have an allergic reaction to vancomycin so I have itchy skin, sore throat/burning/itching, numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",vancomycin,31,41,DRUG,0.998339
7,"I have an allergic reaction to vancomycin so I have itchy skin, sore throat/burning/itching, numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",itchy skin,52,62,ADE,0.924099
8,"I have an allergic reaction to vancomycin so I have itchy skin, sore throat/burning/itching, numbness of tongue a

#### Example 2

In [31]:
process_s3_json_output_and_save(validation_json_file_name2)

,document,ner_chunk,begin,end,ner_label,confidence
0,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, Xenaderm to wounds topically b.i.d., Lantus 40 units subcutaneously at bedtime, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. daily, Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., Percocet 5/325 mg 2 tablets q.4 h. p.r.n., magnesium citrate 1 bottle p.o. p.r.n., sliding scale coverage insulin, Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d.",diabetes,77,85,Disease_Syndrome_Disorder,0.604111
1,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, Xenaderm to wounds topically b.i.d., Lantus 40 units subcutaneously at bedtime, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. daily, Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., Percocet 5/325 mg 2 tablets q.4 h. p.r.n., magnesium citrate 1 bottle p.o. p.r.n., sliding scale coverage insulin, Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d.",type 2,87,93,Disease_Syndrome_Disorder,0.698794
2,"The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. Sh

#### Example 3

In [32]:
process_s3_json_output_and_save(validation_json_file_name3)

,document,ner_chunk,begin,end,ner_label,confidence
0,"I have an allergic reaction to vancomycin so I have itchy skin , sore throat/burning/itching , numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",allergic reaction,10,27,diagnosis,0.687882
1,"I have an allergic reaction to vancomycin so I have itchy skin , sore throat/burning/itching , numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",itchy,54,59,symptom,0.994860
2,"I have an allergic reaction to vancomycin so I have itchy skin , sore throat/burning/itching , numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",skin,60,64,body part,0.995819
3,"I have an allergic reaction to vancomycin so I have itchy skin , sore throat/burning/itching , numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",sore,67,71,symptom,0.984152
4,"I have an allergic reaction to vancomycin so I have itchy skin , sore throat/burning/itching , numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",throat,72,78,body part,0.985427
5,"I have an allergic reaction to vancomycin so I have itchy skin , sore throat/burning/itching , numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",burning,79,86,symptom,0.989564
6,"I have an allergic reaction to vancomycin so I have itchy skin , sore throat/burning/itching , numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",itching,87,94,symptom,0.994643
7,"I have an allergic reaction to vancomycin so I have itchy skin , sore throat/burning/itching , numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",numbness,97,105,symptom,0.957343
8,"I have an allergic reaction to vancomycin so I have itchy skin , sore throat/burning/itching , numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",tongue,109,115,body part,0.944688
9,"I have an allergic reaction to vancomycin so I have itchy skin , sore throat/burning/itching , numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.",gums,121,125,body part,0.988204


### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [34]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

#### Example 1

In [35]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name1)

{"predictions": [{"ner_chunk": "blood clots", "begin": 27, "end": 38, "ner_label": "ADE", "confidence": 0.6221432089805603}, {"ner_chunk": "Voltaren", "begin": 49, "end": 57, "ner_label": "DRUG", "confidence": 0.9907069206237793}, {"ner_chunk": "blood clots", "begin": 125, "end": 136, "ner_label": "ADE", "confidence": 0.7485202550888062}, {"ner_chunk": "voltaren", "begin": 302, "end": 310, "ner_label": "DRUG", "confidence": 0.9585478901863098}, {"ner_chunk": "tylonol", "begin": 544, "end": 551, "ner_label": "DRUG", "confidence": 0.8553090691566467}]}
{"predictions": [{"ner_chunk": "allergic reaction", "begin": 10, "end": 27, "ner_label": "ADE", "confidence": 0.9307523965835571}, {"ner_chunk": "vancomycin", "begin": 31, "end": 41, "ner_label": "DRUG", "confidence": 0.998339056968689}, {"ner_chunk": "itchy skin", "begin": 52, "end": 62, "ner_label": "ADE", "confidence": 0.9240986704826355}, {"ner_chunk": "sore throat", "begin": 64, "end": 75, "ner_label": "ADE", "confidence": 0.907771229

#### Example 2

In [36]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name2)

{"predictions": [{"ner_chunk": "diabetes", "begin": 77, "end": 85, "ner_label": "Disease_Syndrome_Disorder", "confidence": 0.6041108965873718}, {"ner_chunk": "type 2", "begin": 87, "end": 93, "ner_label": "Disease_Syndrome_Disorder", "confidence": 0.6987941265106201}, {"ner_chunk": "coronary artery disease", "begin": 95, "end": 118, "ner_label": "Disease_Syndrome_Disorder", "confidence": 0.8235759735107422}, {"ner_chunk": "chronic renal insufficiency", "begin": 120, "end": 147, "ner_label": "Disease_Syndrome_Disorder", "confidence": 0.8366949558258057}, {"ner_chunk": "peripheral vascular disease", "begin": 149, "end": 176, "ner_label": "Disease_Syndrome_Disorder", "confidence": 0.8868952393531799}, {"ner_chunk": "diabetes", "begin": 196, "end": 204, "ner_label": "Disease_Syndrome_Disorder", "confidence": 0.9246309399604797}, {"ner_chunk": "paraplegia", "begin": 287, "end": 297, "ner_label": "Disease_Syndrome_Disorder", "confidence": 0.5937107801437378}, {"ner_chunk": "lower extremities

#### Example 3

In [37]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name3)

{"predictions": [{"ner_chunk": "allergic reaction", "begin": 10, "end": 27, "ner_label": "diagnosis", "confidence": 0.6878823637962341}, {"ner_chunk": "itchy", "begin": 54, "end": 59, "ner_label": "symptom", "confidence": 0.9948595762252808}, {"ner_chunk": "skin", "begin": 60, "end": 64, "ner_label": "body part", "confidence": 0.995819091796875}, {"ner_chunk": "sore", "begin": 67, "end": 71, "ner_label": "symptom", "confidence": 0.98415207862854}, {"ner_chunk": "throat", "begin": 72, "end": 78, "ner_label": "body part", "confidence": 0.9854265451431274}, {"ner_chunk": "burning", "begin": 79, "end": 86, "ner_label": "symptom", "confidence": 0.9895642995834351}, {"ner_chunk": "itching", "begin": 87, "end": 94, "ner_label": "symptom", "confidence": 0.9946433305740356}, {"ner_chunk": "numbness", "begin": 97, "end": 105, "ner_label": "symptom", "confidence": 0.9573434591293335}, {"ner_chunk": "tongue", "begin": 109, "end": 115, "ner_label": "body part", "confidence": 0.9446883797645569}, {"

In [38]:
model.delete_model()

INFO:sagemaker:Deleting model with name: zero-shot-ner-2024-04-25-00-26-29-786


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

